In [ ]:
print("Getting notebook info")

In [9]:
!pip install rt-utils pydicom pandas numpy scipy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 37.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 32.0 MB/s eta 0:00:0000:01


In [10]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt

# Load existing RT Struct. Requires the series path and existing RT Struct path
rtstruct = RTStructBuilder.create_from(
  dicom_series_path="/projekty/Odbytnice/Data_Organized/172543/CT_DICOM", 
  rt_struct_path="/projekty/Odbytnice/Data_Organized/172543/TotalSegmentator.dcm"
)

In [11]:
# View all of the ROI names from within the image
print(rtstruct.get_roi_names())

['spleen', 'kidney_right', 'kidney_left', 'gallbladder', 'liver', 'stomach', 'pancreas', 'adrenal_gland_right', 'adrenal_gland_left', 'lung_lower_lobe_right', 'small_bowel', 'duodenum', 'colon', 'urinary_bladder', 'kidney_cyst_left', 'sacrum', 'vertebrae_S1', 'vertebrae_L5', 'vertebrae_L4', 'vertebrae_L3', 'vertebrae_L2', 'vertebrae_L1', 'vertebrae_T12', 'vertebrae_T11', 'aorta', 'inferior_vena_cava', 'portal_vein_and_splenic_vein', 'iliac_artery_left', 'iliac_artery_right', 'iliac_vena_left', 'iliac_vena_right', 'humerus_right', 'femur_left', 'femur_right', 'hip_left', 'hip_right', 'spinal_cord', 'gluteus_maximus_left', 'gluteus_maximus_right', 'gluteus_medius_left', 'gluteus_medius_right', 'gluteus_minimus_left', 'gluteus_minimus_right', 'autochthon_left', 'autochthon_right', 'iliopsoas_left', 'iliopsoas_right', 'skull', 'rib_left_8', 'rib_left_9', 'rib_left_10', 'rib_left_11', 'rib_left_12', 'rib_right_8', 'rib_right_9', 'rib_right_10', 'rib_right_11', 'rib_right_12', 'costal_cartil

In [32]:
import numpy as np
import scipy.ndimage as ndi
from scipy.ndimage import binary_erosion, binary_dilation
from rt_utils import RTStructBuilder
from typing import Union, List, Tuple, Optional
import warnings
import cv2 as cv

class RTStructExtended:
    """
    Extended RT Struct functionality for volume calculations, boolean operations,
    and morphological operations with directional control.
    """
    
    def __init__(self, rtstruct):
        """
        Initialize with an existing RTStruct object from rt-utils
        
        Parameters:
        -----------
        rtstruct : RTStruct
            RTStruct object created with RTStructBuilder
        """
        self.rtstruct = rtstruct
        self.ds = rtstruct.ds  # Access to DICOM dataset
        self.series_data = rtstruct.series_data  # Access to series data
        self.pixel_spacing = self._get_pixel_spacing()
        self.slice_thickness = self._get_slice_thickness()
        
        # Store fixed ROI masks in memory
        self._fixed_masks = {}  # roi_name -> fixed_mask
        
    @classmethod
    def create_from(cls, dicom_series_path: str, rt_struct_path: str):
        """
        Create RTStructExtended from DICOM series and RT Struct file
        
        Parameters:
        -----------
        dicom_series_path : str
            Path to DICOM series directory
        rt_struct_path : str
            Path to RT Struct file
            
        Returns:
        --------
        RTStructExtended
        """
        rtstruct = RTStructBuilder.create_from(
            dicom_series_path=dicom_series_path,
            rt_struct_path=rt_struct_path
        )
        return cls(rtstruct)
    
    def _get_pixel_spacing(self) -> Tuple[float, float]:
        """Get pixel spacing from DICOM metadata"""
        if hasattr(self.rtstruct, 'series_data') and len(self.rtstruct.series_data) > 0:
            pixel_spacing = self.rtstruct.series_data[0].PixelSpacing
            return float(pixel_spacing[0]), float(pixel_spacing[1])
        else:
            warnings.warn("Could not retrieve pixel spacing, using default 1.0mm")
            return 1.0, 1.0
    
    def _get_slice_thickness(self) -> float:
        """Get slice thickness from DICOM metadata"""
        if hasattr(self.rtstruct, 'series_data') and len(self.rtstruct.series_data) > 0:
            if hasattr(self.rtstruct.series_data[0], 'SliceThickness'):
                return float(self.rtstruct.series_data[0].SliceThickness)
            else:
                # Calculate from Image Position Patient if SliceThickness not available
                if len(self.rtstruct.series_data) > 1:
                    pos1 = self.rtstruct.series_data[0].ImagePositionPatient
                    pos2 = self.rtstruct.series_data[1].ImagePositionPatient
                    thickness = abs(pos1[2] - pos2[2])
                    return thickness
        warnings.warn("Could not retrieve slice thickness, using default 3.0mm")
        return 3.0
    
    def get_voxel_volume(self) -> float:
        """
        Calculate volume of a single voxel in mm³
        
        Returns:
        --------
        float
            Volume of single voxel in mm³
        """
        return self.pixel_spacing[0] * self.pixel_spacing[1] * self.slice_thickness
    
    def _get_roi_mask_safe(self, roi_name: str, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Safely get ROI mask with error handling and optional auto-fixing
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        auto_fix : bool
            Whether to attempt fixing if standard method fails
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask or None if error occurred
        """
        # First check if we have a fixed version in memory
        if roi_name in self._fixed_masks:
            return self._fixed_masks[roi_name]
        
        # Try the standard method
        try:
            return self.rtstruct.get_roi_mask_by_name(roi_name)
        except Exception as e:
            if auto_fix:
                print(f"Standard method failed for '{roi_name}', attempting to fix...")
                fixed_mask = self.get_fixed_roi_mask(roi_name)
                if fixed_mask is not None:
                    # Store the fixed mask in memory
                    self._fixed_masks[roi_name] = fixed_mask
                    return fixed_mask
            else:
                print(f"Error getting mask for ROI '{roi_name}': {e}")
            return None
    
    def is_roi_valid(self, roi_name: str) -> bool:
        """
        Check if ROI can be processed without errors
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
            
        Returns:
        --------
        bool
            True if ROI is valid and can be processed
        """
        mask = self._get_roi_mask_safe(roi_name)
        return mask is not None
    
    def get_valid_rois(self) -> List[str]:
        """
        Get list of ROIs that can be processed without errors
        
        Returns:
        --------
        List[str]
            List of valid ROI names
        """
        all_rois = self.rtstruct.get_roi_names()
        valid_rois = []
        
        for roi_name in all_rois:
            if self.is_roi_valid(roi_name):
                valid_rois.append(roi_name)
            else:
                print(f"ROI '{roi_name}' has corrupted or invalid contour data")
        
        return valid_rois
    
    def diagnose_roi_issues(self) -> dict:
        """
        Diagnose issues with ROIs in the RT Struct
        
        Returns:
        --------
        dict
            Dictionary with ROI diagnostics
        """
        all_rois = self.rtstruct.get_roi_names()
        diagnostics = {
            'total_rois': len(all_rois),
            'valid_rois': [],
            'invalid_rois': [],
            'roi_details': {}
        }
        
        for roi_name in all_rois:
            try:
                mask = self.rtstruct.get_roi_mask_by_name(roi_name)
                if mask is not None:
                    volume = np.sum(mask) * self.get_voxel_volume()
                    diagnostics['valid_rois'].append(roi_name)
                    diagnostics['roi_details'][roi_name] = {
                        'status': 'valid',
                        'volume_mm3': volume,
                        'volume_ml': volume / 1000.0,
                        'voxel_count': int(np.sum(mask)),
                        'shape': mask.shape
                    }
                else:
                    diagnostics['invalid_rois'].append(roi_name)
                    diagnostics['roi_details'][roi_name] = {
                        'status': 'invalid',
                        'error': 'Mask is None'
                    }
            except Exception as e:
                diagnostics['invalid_rois'].append(roi_name)
                diagnostics['roi_details'][roi_name] = {
                    'status': 'invalid',
                    'error': str(e)
                }
        
        return diagnostics
    
    def print_roi_summary(self):
        """
        Print a summary of ROI status and volumes
        """
        diagnostics = self.diagnose_roi_issues()
        
        print(f"\n=== RT Struct ROI Summary ===")
        print(f"Total ROIs: {diagnostics['total_rois']}")
        print(f"Valid ROIs: {len(diagnostics['valid_rois'])}")
        print(f"Invalid ROIs: {len(diagnostics['invalid_rois'])}")
        
        if diagnostics['invalid_rois']:
            print(f"\nInvalid ROIs (corrupted contour data):")
            for roi_name in diagnostics['invalid_rois']:
                error = diagnostics['roi_details'][roi_name]['error']
                print(f"  - {roi_name}: {error}")
        
        print(f"\nValid ROIs with volumes:")
        valid_rois_sorted = sorted(diagnostics['valid_rois'], 
                                 key=lambda x: diagnostics['roi_details'][x]['volume_ml'], 
                                 reverse=True)
        for roi_name in valid_rois_sorted:
            details = diagnostics['roi_details'][roi_name]
            print(f"  - {roi_name}: {details['volume_ml']:.2f} ml ({details['voxel_count']} voxels)")
        
        print(f"\nPixel spacing: {self.pixel_spacing[0]:.3f} x {self.pixel_spacing[1]:.3f} mm")
        print(f"Slice thickness: {self.slice_thickness:.3f} mm")
        print(f"Voxel volume: {self.get_voxel_volume():.6f} mm³")
        print("=" * 35)
    
    # Contour Fixing Methods
    def _get_image_geometry(self):
        """Get image geometry parameters for coordinate transformation"""
        if not hasattr(self, '_image_geometry'):
            self._image_geometry = {
                'pixel_spacing': [float(x) for x in self.series_data[0].PixelSpacing],
                'positions': np.array([[float(x) for x in slice_data.ImagePositionPatient] 
                                     for slice_data in self.series_data]),
                'orientation': np.array([float(x) for x in self.series_data[0].ImageOrientationPatient]).reshape(2, 3)
            }
        return self._image_geometry
    
    def _world_to_pixel(self, world_coords, slice_index):
        """Convert world coordinates to pixel coordinates for a specific slice"""
        geom = self._get_image_geometry()
        image_position = geom['positions'][slice_index]
        
        # Translate to image coordinate system
        translated = world_coords - image_position
        
        # Project onto image axes
        row_cosines = geom['orientation'][0]
        col_cosines = geom['orientation'][1]
        
        x_coords = np.dot(translated, row_cosines) / geom['pixel_spacing'][0]
        y_coords = np.dot(translated, col_cosines) / geom['pixel_spacing'][1]
        
        pixel_coords = np.column_stack([x_coords, y_coords])
        return pixel_coords.astype(np.int32)
    
    def _fix_contour_data(self, contour_data, min_points=3):
        """Fix contour coordinate data formatting issues"""
        if len(contour_data) < min_points * 3:
            return None
        
        try:
            points = np.array(contour_data, dtype=float).reshape(-1, 3)
            
            if np.any(np.isnan(points)) or np.any(np.isinf(points)):
                return None
            
            if len(points) < min_points:
                return None
            
            # Close contour if not already closed
            if not np.allclose(points[0], points[-1], atol=1e-6):
                points = np.vstack([points, points[0]])
            
            return points
        except Exception:
            return None
    
    def _get_contour_sequence_by_roi_number(self, roi_number: int):
        """Get contour sequence for ROI number"""
        try:
            for roi_contour in self.ds.ROIContourSequence:
                if roi_contour.ReferencedROINumber == roi_number:
                    if hasattr(roi_contour, 'ContourSequence'):
                        return roi_contour.ContourSequence
            return None
        except Exception:
            return None
    
    def _get_slice_contours(self, roi_name, slice_uid):
        """Get fixed contours for a specific slice"""
        # Find ROI number
        roi_number = None
        for structure_roi in self.ds.StructureSetROISequence:
            if structure_roi.ROIName == roi_name:
                roi_number = structure_roi.ROINumber
                break
        
        if roi_number is None:
            return []
        
        contour_sequence = self._get_contour_sequence_by_roi_number(roi_number)
        if not contour_sequence:
            return []
        
        # Extract contours for this slice
        slice_contours = []
        for contour in contour_sequence:
            try:
                if (hasattr(contour, 'ContourImageSequence') and 
                    len(contour.ContourImageSequence) > 0):
                    
                    contour_slice_uid = contour.ContourImageSequence[0].ReferencedSOPInstanceUID
                    
                    if contour_slice_uid == slice_uid and hasattr(contour, 'ContourData'):
                        fixed_contour = self._fix_contour_data(contour.ContourData)
                        if fixed_contour is not None:
                            slice_contours.append(fixed_contour)
            except Exception:
                continue
        
        return slice_contours
    
    def get_fixed_roi_mask(self, roi_name: str) -> Union[np.ndarray, None]:
        """
        Get ROI mask with contour fixing for failed ROIs
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI to fix
            
        Returns:
        --------
        np.ndarray or None
            3D boolean mask or None if failed
        """
        try:
            # Initialize mask
            rows = self.series_data[0].Rows
            cols = self.series_data[0].Columns  
            num_slices = len(self.series_data)
            mask = np.zeros((rows, cols, num_slices), dtype=bool)
            
            total_contours = 0
            valid_contours = 0
            
            # Process each slice
            for i, slice_data in enumerate(self.series_data):
                slice_uid = slice_data.SOPInstanceUID
                slice_contours = self._get_slice_contours(roi_name, slice_uid)
                
                total_contours += len(slice_contours)
                
                if slice_contours:
                    slice_mask = np.zeros((rows, cols), dtype=np.uint8)
                    
                    for contour_3d in slice_contours:
                        pixel_coords = self._world_to_pixel(contour_3d, i)
                        
                        # Clip to image bounds
                        pixel_coords[:, 0] = np.clip(pixel_coords[:, 0], 0, cols - 1)
                        pixel_coords[:, 1] = np.clip(pixel_coords[:, 1], 0, rows - 1)
                        
                        if len(pixel_coords) >= 3:
                            cv.fillPoly(slice_mask, [pixel_coords], 1)
                            valid_contours += 1
                    
                    mask[:, :, i] = slice_mask.astype(bool)
            
            total_voxels = np.sum(mask)
            
            if total_voxels > 0:
                return mask
            else:
                return None
                
        except Exception as e:
            print(f"Error fixing {roi_name}: {e}")
            return None
    
    def fix_roi(self, roi_name: str, color: List[int] = None) -> Union[np.ndarray, None]:
        """
        Fix an ROI with contour issues and store the result in memory
        
        Parameters:
        -----------
        roi_name : str
            ROI name to fix
        color : List[int], optional
            RGB color (for future use)
            
        Returns:
        --------
        np.ndarray or None
            Fixed mask, or None if failed
        """
        print(f"🔧 Fixing ROI: {roi_name}")
        
        fixed_mask = self.get_fixed_roi_mask(roi_name)
        
        if fixed_mask is not None and np.sum(fixed_mask) > 0:
            voxel_count = np.sum(fixed_mask)
            volume_ml = voxel_count * self.get_voxel_volume() / 1000.0
            print(f"✅ Fixed: {roi_name} → {voxel_count:,} voxels ({volume_ml:.2f} ml)")
            
            # Always store in memory for this simplified approach
            self._fixed_masks[roi_name] = fixed_mask
            print(f"   Stored in memory: {roi_name}")
            
            return fixed_mask
        else:
            print(f"❌ Failed to fix: {roi_name}")
            return None
    
    def fix_failed_rois(self, failed_roi_names: List[str] = None, 
                       auto_detect: bool = True) -> dict:
        """
        Fix multiple failed ROIs automatically
        
        Parameters:
        -----------
        failed_roi_names : List[str], optional
            List of ROI names to fix. If None and auto_detect=True, 
            will detect failed ROIs automatically
        auto_detect : bool
            Whether to automatically detect failed ROIs
            
        Returns:
        --------
        dict
            Summary of results
        """
        if failed_roi_names is None and auto_detect:
            print("🔍 Auto-detecting failed ROIs...")
            all_rois = self.rtstruct.get_roi_names()
            failed_roi_names = []
            
            for roi_name in all_rois:
                if not self.is_roi_valid(roi_name):
                    failed_roi_names.append(roi_name)
            
            print(f"Found {len(failed_roi_names)} failed ROIs")
        
        if not failed_roi_names:
            print("No ROIs to fix")
            return {'successful': [], 'failed': [], 'total_voxels': 0}
        
        print(f"\n🔧 FIXING {len(failed_roi_names)} FAILED ROIS")
        print("=" * 50)
        
        results = {
            'successful': [],
            'failed': [],
            'total_voxels': 0
        }
        
        for roi_name in failed_roi_names:
            fixed_mask = self.fix_roi(roi_name)
            
            if fixed_mask is not None:
                results['successful'].append(roi_name)
                results['total_voxels'] += np.sum(fixed_mask)
            else:
                results['failed'].append(roi_name)
        
        print(f"\n📊 RESULTS:")
        print(f"✅ Successfully recovered: {len(results['successful'])}/{len(failed_roi_names)} ROIs")
        print(f"❌ Still failed: {len(results['failed'])} ROIs")
        print(f"🔢 Total recovered voxels: {results['total_voxels']:,}")
        print(f"💾 Fixed ROIs stored in memory and ready to use")
        
        if results['successful']:
            print(f"\n✅ RECOVERED ROIS:")
            for roi in results['successful']:
                print(f"  - {roi}")
        
        return results
    
    def get_roi_mask_with_fixing(self, roi_name: str) -> Union[np.ndarray, None]:
        """
        Get ROI mask, automatically trying to fix if standard method fails
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask
        """
        return self._get_roi_mask_safe(roi_name, auto_fix=True)
    
    def get_fixed_rois(self) -> List[str]:
        """
        Get list of ROIs that have been fixed and stored in memory
        
        Returns:
        --------
        List[str]
            List of fixed ROI names
        """
        return list(self._fixed_masks.keys())
    
    def clear_fixed_rois(self):
        """Clear all fixed ROIs from memory"""
        self._fixed_masks.clear()
        print("Cleared all fixed ROIs from memory")
    
    def is_roi_fixed(self, roi_name: str) -> bool:
        """
        Check if an ROI has been fixed and is stored in memory
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
            
        Returns:
        --------
        bool
            True if ROI is fixed and stored in memory
        """
        return roi_name in self._fixed_masks
    
    def calculate_volume(self, roi_name: str, auto_fix: bool = False) -> float:
        """
        Calculate volume of an ROI in mm³
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        auto_fix : bool
            Whether to attempt fixing if ROI fails with standard method
            
        Returns:
        --------
        float
            Volume in mm³, or 0 if ROI is invalid
        """
        mask = self._get_roi_mask_safe(roi_name, auto_fix=auto_fix)
        if mask is None:
            return 0.0
        
        voxel_count = np.sum(mask)
        voxel_volume = self.get_voxel_volume()
        return voxel_count * voxel_volume
    
    def calculate_volume_ml(self, roi_name: str, auto_fix: bool = False) -> float:
        """
        Calculate volume of an ROI in ml (cm³)
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        auto_fix : bool
            Whether to attempt fixing if ROI fails with standard method
            
        Returns:
        --------
        float
            Volume in ml (cm³)
        """
        return self.calculate_volume(roi_name, auto_fix=auto_fix) / 1000.0
    
    def get_volumes_summary(self, roi_names: Optional[List[str]] = None) -> dict:
        """
        Get volume summary for multiple ROIs
        
        Parameters:
        -----------
        roi_names : List[str], optional
            List of ROI names. If None, calculates for all valid ROIs
            
        Returns:
        --------
        dict
            Dictionary with ROI names as keys and volumes (mm³, ml) as values
        """
        if roi_names is None:
            roi_names = self.get_valid_rois()
        
        summary = {}
        for roi_name in roi_names:
            if self.is_roi_valid(roi_name):
                vol_mm3 = self.calculate_volume(roi_name)
                vol_ml = vol_mm3 / 1000.0
                summary[roi_name] = {
                    'volume_mm3': vol_mm3,
                    'volume_ml': vol_ml
                }
            else:
                print(f"Skipping invalid ROI: {roi_name}")
                
        return summary
    
    # Boolean Operations
    def union(self, roi1: str, roi2: str, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Perform union (OR) operation on two ROIs
        
        Parameters:
        -----------
        roi1, roi2 : str
            Names of the ROIs
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask of the union, or None if either ROI is invalid
        """
        mask1 = self._get_roi_mask_safe(roi1, auto_fix=auto_fix)
        mask2 = self._get_roi_mask_safe(roi2, auto_fix=auto_fix)
        
        if mask1 is None or mask2 is None:
            print(f"Cannot perform union: invalid ROI data for '{roi1}' or '{roi2}'")
            return None
            
        return np.logical_or(mask1, mask2)
    
    def intersection(self, roi1: str, roi2: str, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Perform intersection (AND) operation on two ROIs
        
        Parameters:
        -----------
        roi1, roi2 : str
            Names of the ROIs
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask of the intersection, or None if either ROI is invalid
        """
        mask1 = self._get_roi_mask_safe(roi1, auto_fix=auto_fix)
        mask2 = self._get_roi_mask_safe(roi2, auto_fix=auto_fix)
        
        if mask1 is None or mask2 is None:
            print(f"Cannot perform intersection: invalid ROI data for '{roi1}' or '{roi2}'")
            return None
            
        return np.logical_and(mask1, mask2)
    
    def difference(self, roi1: str, roi2: str, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Perform difference operation (roi1 - roi2)
        
        Parameters:
        -----------
        roi1, roi2 : str
            Names of the ROIs
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask of roi1 minus roi2, or None if either ROI is invalid
        """
        mask1 = self._get_roi_mask_safe(roi1, auto_fix=auto_fix)
        mask2 = self._get_roi_mask_safe(roi2, auto_fix=auto_fix)
        
        if mask1 is None or mask2 is None:
            print(f"Cannot perform difference: invalid ROI data for '{roi1}' or '{roi2}'")
            return None
            
        return np.logical_and(mask1, np.logical_not(mask2))
    
    def xor(self, roi1: str, roi2: str, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Perform exclusive OR operation on two ROIs
        
        Parameters:
        -----------
        roi1, roi2 : str
            Names of the ROIs
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask of the XOR, or None if either ROI is invalid
        """
        mask1 = self._get_roi_mask_safe(roi1, auto_fix=auto_fix)
        mask2 = self._get_roi_mask_safe(roi2, auto_fix=auto_fix)
        
        if mask1 is None or mask2 is None:
            print(f"Cannot perform XOR: invalid ROI data for '{roi1}' or '{roi2}'")
            return None
            
        return np.logical_xor(mask1, mask2)
    
    def multi_union(self, roi_names: List[str], auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Perform union operation on multiple ROIs
        
        Parameters:
        -----------
        roi_names : List[str]
            List of ROI names
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        np.ndarray or None
            Boolean mask of the union of all valid ROIs, or None if no valid ROIs
        """
        if not roi_names:
            raise ValueError("roi_names cannot be empty")
        
        valid_masks = []
        for roi_name in roi_names:
            mask = self._get_roi_mask_safe(roi_name, auto_fix=auto_fix)
            if mask is not None:
                valid_masks.append(mask)
            else:
                print(f"Skipping invalid ROI: {roi_name}")
        
        if not valid_masks:
            print("No valid ROIs found for union operation")
            return None
        
        result = valid_masks[0]
        for mask in valid_masks[1:]:
            result = np.logical_or(result, mask)
        
        return result
    
    # Morphological Operations
    def _mm_to_voxels(self, distance_mm: float, axis: int) -> int:
        """
        Convert distance in mm to voxels for given axis
        
        Parameters:
        -----------
        distance_mm : float
            Distance in millimeters
        axis : int
            Axis (0=row/y, 1=col/x, 2=slice/z)
            
        Returns:
        --------
        int
            Distance in voxels
        """
        if axis == 0:  # rows (y-direction)
            return int(np.round(distance_mm / self.pixel_spacing[1]))
        elif axis == 1:  # columns (x-direction)
            return int(np.round(distance_mm / self.pixel_spacing[0]))
        elif axis == 2:  # slices (z-direction)
            return int(np.round(distance_mm / self.slice_thickness))
        else:
            raise ValueError("axis must be 0, 1, or 2")
    
    def expand_isotropic(self, roi_name: str, distance_mm: float, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Expand ROI isotropically by specified distance
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        distance_mm : float
            Distance to expand in mm
        auto_fix : bool
            Whether to attempt fixing if ROI fails with standard method
            
        Returns:
        --------
        np.ndarray or None
            Expanded boolean mask, or None if ROI is invalid
        """
        mask = self._get_roi_mask_safe(roi_name, auto_fix=auto_fix)
        if mask is None:
            print(f"Cannot expand ROI '{roi_name}': invalid ROI data")
            return None
        
        # Calculate structuring element size for each axis
        size_y = self._mm_to_voxels(distance_mm, 0)
        size_x = self._mm_to_voxels(distance_mm, 1)
        size_z = self._mm_to_voxels(distance_mm, 2)
        
        # Create ellipsoidal structuring element
        struct_elem = np.zeros((2*size_y+1, 2*size_x+1, 2*size_z+1))
        center = (size_y, size_x, size_z)
        
        for i in range(struct_elem.shape[0]):
            for j in range(struct_elem.shape[1]):
                for k in range(struct_elem.shape[2]):
                    dist_sq = ((i-center[0])/size_y)**2 + ((j-center[1])/size_x)**2 + ((k-center[2])/size_z)**2
                    if dist_sq <= 1:
                        struct_elem[i, j, k] = 1
        
        return binary_dilation(mask, structure=struct_elem)
    
    def contract_isotropic(self, roi_name: str, distance_mm: float, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Contract ROI isotropically by specified distance
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        distance_mm : float
            Distance to contract in mm
        auto_fix : bool
            Whether to attempt fixing if ROI fails with standard method
            
        Returns:
        --------
        np.ndarray or None
            Contracted boolean mask, or None if ROI is invalid
        """
        mask = self._get_roi_mask_safe(roi_name, auto_fix=auto_fix)
        if mask is None:
            print(f"Cannot contract ROI '{roi_name}': invalid ROI data")
            return None
        
        # Calculate structuring element size for each axis
        size_y = self._mm_to_voxels(distance_mm, 0)
        size_x = self._mm_to_voxels(distance_mm, 1)
        size_z = self._mm_to_voxels(distance_mm, 2)
        
        # Create ellipsoidal structuring element
        struct_elem = np.zeros((2*size_y+1, 2*size_x+1, 2*size_z+1))
        center = (size_y, size_x, size_z)
        
        for i in range(struct_elem.shape[0]):
            for j in range(struct_elem.shape[1]):
                for k in range(struct_elem.shape[2]):
                    dist_sq = ((i-center[0])/size_y)**2 + ((j-center[1])/size_x)**2 + ((k-center[2])/size_z)**2
                    if dist_sq <= 1:
                        struct_elem[i, j, k] = 1
        
        return binary_erosion(mask, structure=struct_elem)
    
    def expand_directional(self, roi_name: str, 
                          x_mm: float = 0, y_mm: float = 0, z_mm: float = 0, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Expand ROI by different amounts in different directions
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        x_mm, y_mm, z_mm : float
            Distance to expand in each direction (mm)
        auto_fix : bool
            Whether to attempt fixing if ROI fails with standard method
            
        Returns:
        --------
        np.ndarray or None
            Expanded boolean mask, or None if ROI is invalid
        """
        mask = self._get_roi_mask_safe(roi_name, auto_fix=auto_fix)
        if mask is None:
            print(f"Cannot expand ROI '{roi_name}': invalid ROI data")
            return None
        
        # Convert to voxels
        x_voxels = self._mm_to_voxels(abs(x_mm), 1) if x_mm != 0 else 0
        y_voxels = self._mm_to_voxels(abs(y_mm), 0) if y_mm != 0 else 0
        z_voxels = self._mm_to_voxels(abs(z_mm), 2) if z_mm != 0 else 0
        
        # Create rectangular structuring element
        struct_shape = (2*y_voxels+1, 2*x_voxels+1, 2*z_voxels+1)
        struct_elem = np.ones(struct_shape)
        
        if x_mm < 0 or y_mm < 0 or z_mm < 0:
            # If any dimension is negative, use erosion
            return binary_erosion(mask, structure=struct_elem)
        else:
            return binary_dilation(mask, structure=struct_elem)
    
    def create_margin(self, roi_name: str, 
                     anterior_mm: float = 0, posterior_mm: float = 0,
                     left_mm: float = 0, right_mm: float = 0,
                     superior_mm: float = 0, inferior_mm: float = 0, auto_fix: bool = False) -> Union[np.ndarray, None]:
        """
        Create asymmetric margins around ROI
        
        Parameters:
        -----------
        roi_name : str
            Name of the ROI
        anterior_mm, posterior_mm : float
            Margins in anterior/posterior directions
        left_mm, right_mm : float
            Margins in left/right directions  
        superior_mm, inferior_mm : float
            Margins in superior/inferior directions
        auto_fix : bool
            Whether to attempt fixing if ROI fails with standard method
            
        Returns:
        --------
        np.ndarray or None
            ROI with margins applied, or None if ROI is invalid
        """
        mask = self._get_roi_mask_safe(roi_name, auto_fix=auto_fix)
        if mask is None:
            print(f"Cannot create margins for ROI '{roi_name}': invalid ROI data")
            return None
        
        result = mask.copy()
        
        # Apply each direction separately
        directions = [
            (anterior_mm, 0, 1, 0),    # anterior (positive x)
            (-posterior_mm, 0, 1, 0),  # posterior (negative x)
            (left_mm, 1, 0, 0),        # left (positive y)
            (-right_mm, 1, 0, 0),      # right (negative y)
            (superior_mm, 0, 0, 1),    # superior (positive z)
            (-inferior_mm, 0, 0, 1)    # inferior (negative z)
        ]
        
        for distance, x, y, z in directions:
            if distance != 0:
                # Use mask directly instead of roi_name for expansion
                temp_mask = result.copy()
                expanded = self._expand_mask_directional(temp_mask,
                                                       x_mm=x*abs(distance), 
                                                       y_mm=y*abs(distance), 
                                                       z_mm=z*abs(distance))
                if distance > 0 and expanded is not None:
                    result = np.logical_or(result, expanded)
                
        return result
    
    def _expand_mask_directional(self, mask: np.ndarray,
                               x_mm: float = 0, y_mm: float = 0, z_mm: float = 0) -> np.ndarray:
        """
        Helper function to expand a mask by different amounts in different directions
        
        Parameters:
        -----------
        mask : np.ndarray
            Input boolean mask
        x_mm, y_mm, z_mm : float
            Distance to expand in each direction (mm)
            
        Returns:
        --------
        np.ndarray
            Expanded boolean mask
        """
        # Convert to voxels
        x_voxels = self._mm_to_voxels(abs(x_mm), 1) if x_mm != 0 else 0
        y_voxels = self._mm_to_voxels(abs(y_mm), 0) if y_mm != 0 else 0
        z_voxels = self._mm_to_voxels(abs(z_mm), 2) if z_mm != 0 else 0
        
        # Create rectangular structuring element
        struct_shape = (2*y_voxels+1, 2*x_voxels+1, 2*z_voxels+1)
        struct_elem = np.ones(struct_shape)
        
        if x_mm < 0 or y_mm < 0 or z_mm < 0:
            # If any dimension is negative, use erosion
            return binary_erosion(mask, structure=struct_elem)
        else:
            return binary_dilation(mask, structure=struct_elem)
    
    def add_roi_from_mask(self, mask: np.ndarray, roi_name: str, color: Optional[List[int]] = None):
        """
        Add a new ROI to the RT Struct from a boolean mask
        
        Parameters:
        -----------
        mask : np.ndarray
            Boolean mask for the new ROI
        roi_name : str
            Name for the new ROI
        color : List[int], optional
            RGB color for the ROI [R, G, B]
        """
        if color is None:
            color = [255, 0, 0]  # Default red
            
        self.rtstruct.add_roi(mask=mask, color=color, name=roi_name)
    
    def get_overlap_volume(self, roi1: str, roi2: str, auto_fix: bool = False) -> float:
        """
        Calculate overlap volume between two ROIs
        
        Parameters:
        -----------
        roi1, roi2 : str
            Names of the ROIs
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        float
            Overlap volume in mm³, or 0 if either ROI is invalid
        """
        intersection_mask = self.intersection(roi1, roi2, auto_fix=auto_fix)
        if intersection_mask is None:
            return 0.0
        voxel_count = np.sum(intersection_mask)
        return voxel_count * self.get_voxel_volume()
    
    def get_dice_coefficient(self, roi1: str, roi2: str, auto_fix: bool = False) -> float:
        """
        Calculate Dice similarity coefficient between two ROIs
        
        Parameters:
        -----------
        roi1, roi2 : str
            Names of the ROIs
        auto_fix : bool
            Whether to attempt fixing if ROIs fail with standard method
            
        Returns:
        --------
        float
            Dice coefficient (0-1), or 0 if either ROI is invalid
        """
        mask1 = self._get_roi_mask_safe(roi1, auto_fix=auto_fix)
        mask2 = self._get_roi_mask_safe(roi2, auto_fix=auto_fix)
        
        if mask1 is None or mask2 is None:
            print(f"Cannot calculate Dice coefficient: invalid ROI data for '{roi1}' or '{roi2}'")
            return 0.0
        
        intersection = np.sum(np.logical_and(mask1, mask2))
        total = np.sum(mask1) + np.sum(mask2)
        
        if total == 0:
            return 1.0  # Both masks are empty
        
        return 2.0 * intersection / total

# Usage example
if __name__ == "__main__":
    # Create extended RT Struct
    rt_ext = RTStructExtended.create_from(
        dicom_series_path="/projekty/Odbytnice/Data_Organized/172543/CT_DICOM",
        rt_struct_path="/projekty/Odbytnice/Data_Organized/172543/TotalSegmentator.dcm"
    )
    
    # Print comprehensive ROI summary
    rt_ext.print_roi_summary()
    
    # Option 1: Automatically fix all failed ROIs (stores in memory)
    print("\n🔧 AUTOMATIC ROI FIXING")
    fix_results = rt_ext.fix_failed_rois()
    
    # Check which ROIs are now fixed in memory
    fixed_rois = rt_ext.get_fixed_rois()
    print(f"\nFixed ROIs in memory: {len(fixed_rois)}")
    print(f"Original ROI count: {len(rt_ext.rtstruct.get_roi_names())} (unchanged)")
    
    # Option 2: Use auto_fix parameter in operations
    print("\n🔧 USING AUTO-FIX IN OPERATIONS")
    
    # Calculate volume with auto-fixing (seamlessly uses fixed version)
    liver_volume = rt_ext.calculate_volume_ml("liver", auto_fix=True)
    print(f"Liver volume (with auto-fix): {liver_volume:.2f} ml")
    
    # Boolean operations with auto-fixing
    liver_kidney_union = rt_ext.union("liver", "kidney_left", auto_fix=True)
    if liver_kidney_union is not None:
        print("✅ Union of liver and kidney_left successful with auto-fix")
    
    # Morphological operations with auto-fixing
    liver_expanded = rt_ext.expand_isotropic("liver", 5.0, auto_fix=True)
    if liver_expanded is not None:
        print("✅ Liver expansion successful with auto-fix")
    
    # Create PTV with auto-fixing
    ptv = rt_ext.create_margin("liver", 
                              anterior_mm=10, posterior_mm=5,
                              left_mm=7, right_mm=7,
                              superior_mm=10, inferior_mm=10,
                              auto_fix=True)
    if ptv is not None:
        rt_ext.add_roi_from_mask(ptv, "PTV_Liver", color=[255, 100, 100])
        print("✅ PTV creation successful with auto-fix")
    
    # Option 3: Direct usage (fixed ROIs work transparently)
    print("\n✅ DIRECT USAGE - All ROIs now work transparently:")
    
    # After fixing, these calls work seamlessly with original names
    volumes = {}
    test_rois = ['liver', 'kidney_left', 'iliac_artery_left', 'colon']
    
    for roi_name in test_rois:
        if rt_ext.is_roi_fixed(roi_name):
            vol = rt_ext.calculate_volume_ml(roi_name)  # Uses fixed version automatically
            volumes[roi_name] = vol
            print(f"  {roi_name}: {vol:.2f} ml ✅ (from fixed version)")
        else:
            print(f"  {roi_name}: Not fixed")
    
    # Boolean operations work seamlessly
    liver_colon_overlap = rt_ext.intersection("liver", "colon")
    if liver_colon_overlap is not None:
        overlap_vol = np.sum(liver_colon_overlap) * rt_ext.get_voxel_volume() / 1000
        print(f"  liver ∩ colon: {overlap_vol:.2f} ml")
    
    print(f"\n💾 Memory usage: {len(rt_ext.get_fixed_rois())} fixed ROIs stored")
    
    # Save enhanced RT Struct if needed (original structure preserved)
    # rt_ext.rtstruct.save("enhanced_rtstruct.dcm")


=== RT Struct ROI Summary ===
Total ROIs: 59
Valid ROIs: 45
Invalid ROIs: 14

Invalid ROIs (corrupted contour data):
  - kidney_left: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

  - liver: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

  - lung_lower_lobe_right: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

  - colon: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

  - sacrum: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

  - vertebrae_L5: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/

In [33]:
rt_ext = RTStructExtended.create_from(
        dicom_series_path="/projekty/Odbytnice/Data_Organized/172543/CT_DICOM",
        rt_struct_path="/projekty/Odbytnice/Data_Organized/172543/TotalSegmentator.dcm"
    )
rt_ext.fix_failed_rois()

🔍 Auto-detecting failed ROIs...
Error getting mask for ROI 'kidney_left': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'liver': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'lung_lower_lobe_right': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'colon': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'sacrum': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for RO

{'successful': ['kidney_left',
  'liver',
  'lung_lower_lobe_right',
  'colon',
  'sacrum',
  'vertebrae_L5',
  'vertebrae_T12',
  'iliac_artery_left',
  'femur_left',
  'iliopsoas_left',
  'skull',
  'rib_left_12',
  'rib_right_12',
  'costal_cartilages'],
 'failed': [],
 'total_voxels': np.int64(632445)}

In [34]:
rt_ext.get_valid_rois()

['spleen',
 'kidney_right',
 'kidney_left',
 'gallbladder',
 'liver',
 'stomach',
 'pancreas',
 'adrenal_gland_right',
 'adrenal_gland_left',
 'lung_lower_lobe_right',
 'small_bowel',
 'duodenum',
 'colon',
 'urinary_bladder',
 'kidney_cyst_left',
 'sacrum',
 'vertebrae_S1',
 'vertebrae_L5',
 'vertebrae_L4',
 'vertebrae_L3',
 'vertebrae_L2',
 'vertebrae_L1',
 'vertebrae_T12',
 'vertebrae_T11',
 'aorta',
 'inferior_vena_cava',
 'portal_vein_and_splenic_vein',
 'iliac_artery_left',
 'iliac_artery_right',
 'iliac_vena_left',
 'iliac_vena_right',
 'humerus_right',
 'femur_left',
 'femur_right',
 'hip_left',
 'hip_right',
 'spinal_cord',
 'gluteus_maximus_left',
 'gluteus_maximus_right',
 'gluteus_medius_left',
 'gluteus_medius_right',
 'gluteus_minimus_left',
 'gluteus_minimus_right',
 'autochthon_left',
 'autochthon_right',
 'iliopsoas_left',
 'iliopsoas_right',
 'skull',
 'rib_left_8',
 'rib_left_9',
 'rib_left_10',
 'rib_left_11',
 'rib_left_12',
 'rib_right_8',
 'rib_right_9',
 'rib_ri

In [35]:
# Create the two new structures
print("🔧 Creating iliac vessel structures...")

# 1) Create iliac_vess: union of all iliac vessels
iliac_vessel_rois = ['iliac_artery_left', 'iliac_artery_right', 'iliac_vena_left', 'iliac_vena_right']

print(f"\n1️⃣ Creating iliac_vess from: {iliac_vessel_rois}")
iliac_vess_mask = rt_ext.multi_union(iliac_vessel_rois, auto_fix=True)

if iliac_vess_mask is not None:
    # Calculate volume
    vess_volume = np.sum(iliac_vess_mask) * rt_ext.get_voxel_volume() / 1000.0
    print(f"✅ iliac_vess created: {vess_volume:.2f} ml")
    
    # Add to RT struct with blue color
    rt_ext.add_roi_from_mask(iliac_vess_mask, "iliac_vess", color=[0, 0, 255])
    print("   Added iliac_vess to RT struct")
    
    # 2) Create iliac_area: 7mm expansion from iliac_vess
    print(f"\n2️⃣ Creating iliac_area: 7mm expansion from iliac_vess")
    iliac_area_mask = rt_ext.expand_isotropic("iliac_vess", 7.0)
    
    if iliac_area_mask is not None:
        # Calculate volume
        area_volume = np.sum(iliac_area_mask) * rt_ext.get_voxel_volume() / 1000.0
        expansion_volume = area_volume - vess_volume
        print(f"✅ iliac_area created: {area_volume:.2f} ml (expansion: +{expansion_volume:.2f} ml)")
        
        # Add to RT struct with cyan color
        rt_ext.add_roi_from_mask(iliac_area_mask, "iliac_area", color=[0, 255, 255])
        print("   Added iliac_area to RT struct")
        
        print(f"\n📊 SUMMARY:")
        print(f"   iliac_vess: {vess_volume:.2f} ml")
        print(f"   iliac_area: {area_volume:.2f} ml")
        print(f"   Expansion volume: {expansion_volume:.2f} ml")
        print(f"   Total ROIs now: {len(rt_ext.rtstruct.get_roi_names())}")
        
    else:
        print("❌ Failed to create iliac_area expansion")
        
else:
    print("❌ Failed to create iliac_vess union")
    print("   Check if the individual vessel ROIs are available:")
    for roi in iliac_vessel_rois:
        status = "✅ Available" if rt_ext.is_roi_valid(roi) or rt_ext.is_roi_fixed(roi) else "❌ Missing"
        print(f"     {roi}: {status}")

🔧 Creating iliac vessel structures...

1️⃣ Creating iliac_vess from: ['iliac_artery_left', 'iliac_artery_right', 'iliac_vena_left', 'iliac_vena_right']
✅ iliac_vess created: 90.30 ml
   Added iliac_vess to RT struct

2️⃣ Creating iliac_area: 7mm expansion from iliac_vess
✅ iliac_area created: 355.00 ml (expansion: +264.69 ml)
   Added iliac_area to RT struct

📊 SUMMARY:
   iliac_vess: 90.30 ml
   iliac_area: 355.00 ml
   Expansion volume: 264.69 ml
   Total ROIs now: 61


In [36]:
# Create pelvic bones structure
print("🔧 Creating pelvic bones structures...")

# Define pelvic bone ROIs
pelvic_bone_rois = [
    'sacrum',           # Sacrum
    'hip_left',         # Left hip bone (ilium, ischium, pubis)
    'hip_right',        # Right hip bone (ilium, ischium, pubis)
    'vertebrae_S1'# S1 vertebra (part of sacrum)
    #'vertebrae_L5',     # L5 vertebra (lower lumbar, close to pelvis)
]

print(f"\n3️⃣ Creating pelvic_bones from: {pelvic_bone_rois}")

# Check availability of pelvic bone ROIs
print("   Checking availability:")
available_bones = []
for roi in pelvic_bone_rois:
    if rt_ext.is_roi_valid(roi) or rt_ext.is_roi_fixed(roi):
        available_bones.append(roi)
        status = "✅ Available"
    else:
        status = "❌ Missing"
    print(f"     {roi}: {status}")

if available_bones:
    print(f"   Using {len(available_bones)}/{len(pelvic_bone_rois)} available bone ROIs")
    
    # Create union of all available pelvic bones
    pelvic_bones_mask = rt_ext.multi_union(available_bones, auto_fix=True)
    
    if pelvic_bones_mask is not None:
        # Calculate volume
        bones_volume = np.sum(pelvic_bones_mask) * rt_ext.get_voxel_volume() / 1000.0
        print(f"✅ pelvic_bones created: {bones_volume:.2f} ml")
        
        # Add to RT struct with orange color
        rt_ext.add_roi_from_mask(pelvic_bones_mask, "pelvic_bones", color=[255, 165, 0])
        print("   Added pelvic_bones to RT struct")
        
        # 4) Create pelvic_bones_3mm: 3mm expansion from pelvic_bones
        print(f"\n4️⃣ Creating pelvic_bones_3mm: 3mm expansion from pelvic_bones")
        pelvic_bones_3mm_mask = rt_ext.expand_isotropic("pelvic_bones", 3.0)
        
        if pelvic_bones_3mm_mask is not None:
            # Calculate volume
            bones_3mm_volume = np.sum(pelvic_bones_3mm_mask) * rt_ext.get_voxel_volume() / 1000.0
            expansion_volume = bones_3mm_volume - bones_volume
            print(f"✅ pelvic_bones_3mm created: {bones_3mm_volume:.2f} ml (expansion: +{expansion_volume:.2f} ml)")
            
            # Add to RT struct with yellow color
            rt_ext.add_roi_from_mask(pelvic_bones_3mm_mask, "pelvic_bones_3mm", color=[255, 255, 0])
            print("   Added pelvic_bones_3mm to RT struct")
            
            print(f"\n📊 PELVIC BONES SUMMARY:")
            print(f"   pelvic_bones: {bones_volume:.2f} ml")
            print(f"   pelvic_bones_3mm: {bones_3mm_volume:.2f} ml")
            print(f"   3mm expansion volume: {expansion_volume:.2f} ml")
            
        else:
            print("❌ Failed to create pelvic_bones_3mm expansion")
            
    else:
        print("❌ Failed to create pelvic_bones union")
        
else:
    print("❌ No pelvic bone ROIs available")



🔧 Creating pelvic bones structures...

3️⃣ Creating pelvic_bones from: ['sacrum', 'hip_left', 'hip_right', 'vertebrae_S1']
   Checking availability:
     sacrum: ✅ Available
     hip_left: ✅ Available
     hip_right: ✅ Available
     vertebrae_S1: ✅ Available
   Using 4/4 available bone ROIs
✅ pelvic_bones created: 1158.43 ml
   Added pelvic_bones to RT struct

4️⃣ Creating pelvic_bones_3mm: 3mm expansion from pelvic_bones
✅ pelvic_bones_3mm created: 1710.28 ml (expansion: +551.85 ml)
   Added pelvic_bones_3mm to RT struct

📊 PELVIC BONES SUMMARY:
   pelvic_bones: 1158.43 ml
   pelvic_bones_3mm: 1710.28 ml
   3mm expansion volume: 551.85 ml


In [38]:
# Save the enhanced RT struct with all new structures
print("💾 Saving enhanced RT struct...")

# Define the output path
output_path = "/projekty/Odbytnice/Data_Organized/172543/TotalSegmentator_ext.dcm"

try:
    # Save the RT struct with all modifications
    rt_ext.rtstruct.save(output_path)
    
    print(f"✅ Successfully saved enhanced RT struct to:")
    print(f"   {output_path}")
    
    # Provide summary of what was saved
    total_rois = len(rt_ext.rtstruct.get_roi_names())
    fixed_rois = len(rt_ext.get_fixed_rois())
    
    print(f"\n📊 SAVED CONTENT SUMMARY:")
    print(f"   Total ROIs: {total_rois}")
    print(f"   Fixed ROIs in memory: {fixed_rois}")
    
    # List the new structures we created
    new_structures = ["iliac_vess", "iliac_area", "pelvic_bones", "pelvic_bones_3mm"]
    created_structures = []
    
    for struct in new_structures:
        try:
            volume = rt_ext.calculate_volume_ml(struct)
            created_structures.append(f"{struct} ({volume:.1f} ml)")
        except:
            pass
    
    if created_structures:
        print(f"   New structures: {len(created_structures)}")
        for struct in created_structures:
            print(f"     • {struct}")

    # Optional: Verify file size
    import os
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"   File size: {file_size:.1f} MB")
    
except Exception as e:
    print(f"❌ Error saving RT struct: {e}")

💾 Saving enhanced RT struct...
Writing file to /projekty/Odbytnice/Data_Organized/172543/TotalSegmentator_ext.dcm
✅ Successfully saved enhanced RT struct to:
   /projekty/Odbytnice/Data_Organized/172543/TotalSegmentator_ext.dcm

📊 SAVED CONTENT SUMMARY:
   Total ROIs: 63
   Fixed ROIs in memory: 14
   New structures: 4
     • iliac_vess (90.3 ml)
     • iliac_area (355.0 ml)
     • pelvic_bones (1158.4 ml)
     • pelvic_bones_3mm (1710.3 ml)
   File size: 7.6 MB


# Full code and loop

In [ ]:
rt_ext = RTStructExtended.create_from(
        dicom_series_path="/projekty/Odbytnice/Data_Organized/172543/CT_DICOM",
        rt_struct_path="/projekty/Odbytnice/Data_Organized/172543/TotalSegmentator.dcm"
    )
rt_ext.fix_failed_rois()

# Create the two new structures
print("🔧 Creating iliac vessel structures...")

# 1) Create iliac_vess: union of all iliac vessels
iliac_vessel_rois = ['iliac_artery_left', 'iliac_artery_right', 'iliac_vena_left', 'iliac_vena_right']

print(f"\n1️⃣ Creating iliac_vess from: {iliac_vessel_rois}")
iliac_vess_mask = rt_ext.multi_union(iliac_vessel_rois, auto_fix=True)

if iliac_vess_mask is not None:
    # Calculate volume
    vess_volume = np.sum(iliac_vess_mask) * rt_ext.get_voxel_volume() / 1000.0
    print(f"✅ iliac_vess created: {vess_volume:.2f} ml")
    
    # Add to RT struct with blue color
    rt_ext.add_roi_from_mask(iliac_vess_mask, "iliac_vess", color=[0, 0, 255])
    print("   Added iliac_vess to RT struct")
    
    # 2) Create iliac_area: 7mm expansion from iliac_vess
    print(f"\n2️⃣ Creating iliac_area: 7mm expansion from iliac_vess")
    iliac_area_mask = rt_ext.expand_isotropic("iliac_vess", 7.0)
    
    if iliac_area_mask is not None:
        # Calculate volume
        area_volume = np.sum(iliac_area_mask) * rt_ext.get_voxel_volume() / 1000.0
        expansion_volume = area_volume - vess_volume
        print(f"✅ iliac_area created: {area_volume:.2f} ml (expansion: +{expansion_volume:.2f} ml)")
        
        # Add to RT struct with cyan color
        rt_ext.add_roi_from_mask(iliac_area_mask, "iliac_area", color=[0, 255, 255])
        print("   Added iliac_area to RT struct")
        
        print(f"\n📊 SUMMARY:")
        print(f"   iliac_vess: {vess_volume:.2f} ml")
        print(f"   iliac_area: {area_volume:.2f} ml")
        print(f"   Expansion volume: {expansion_volume:.2f} ml")
        print(f"   Total ROIs now: {len(rt_ext.rtstruct.get_roi_names())}")
        
    else:
        print("❌ Failed to create iliac_area expansion")
        
else:
    print("❌ Failed to create iliac_vess union")
    print("   Check if the individual vessel ROIs are available:")
    for roi in iliac_vessel_rois:
        status = "✅ Available" if rt_ext.is_roi_valid(roi) or rt_ext.is_roi_fixed(roi) else "❌ Missing"
        print(f"     {roi}: {status}")


# Create pelvic bones structure
print("🔧 Creating pelvic bones structures...")

# Define pelvic bone ROIs
pelvic_bone_rois = [
    'sacrum',           # Sacrum
    'hip_left',         # Left hip bone (ilium, ischium, pubis)
    'hip_right',        # Right hip bone (ilium, ischium, pubis)
    'vertebrae_S1'# S1 vertebra (part of sacrum)
    #'vertebrae_L5',     # L5 vertebra (lower lumbar, close to pelvis)
]

print(f"\n3️⃣ Creating pelvic_bones from: {pelvic_bone_rois}")

# Check availability of pelvic bone ROIs
print("   Checking availability:")
available_bones = []
for roi in pelvic_bone_rois:
    if rt_ext.is_roi_valid(roi) or rt_ext.is_roi_fixed(roi):
        available_bones.append(roi)
        status = "✅ Available"
    else:
        status = "❌ Missing"
    print(f"     {roi}: {status}")

if available_bones:
    print(f"   Using {len(available_bones)}/{len(pelvic_bone_rois)} available bone ROIs")
    
    # Create union of all available pelvic bones
    pelvic_bones_mask = rt_ext.multi_union(available_bones, auto_fix=True)
    
    if pelvic_bones_mask is not None:
        # Calculate volume
        bones_volume = np.sum(pelvic_bones_mask) * rt_ext.get_voxel_volume() / 1000.0
        print(f"✅ pelvic_bones created: {bones_volume:.2f} ml")
        
        # Add to RT struct with orange color
        rt_ext.add_roi_from_mask(pelvic_bones_mask, "pelvic_bones", color=[255, 165, 0])
        print("   Added pelvic_bones to RT struct")
        
        # 4) Create pelvic_bones_3mm: 3mm expansion from pelvic_bones
        print(f"\n4️⃣ Creating pelvic_bones_3mm: 3mm expansion from pelvic_bones")
        pelvic_bones_3mm_mask = rt_ext.expand_isotropic("pelvic_bones", 3.0)
        
        if pelvic_bones_3mm_mask is not None:
            # Calculate volume
            bones_3mm_volume = np.sum(pelvic_bones_3mm_mask) * rt_ext.get_voxel_volume() / 1000.0
            expansion_volume = bones_3mm_volume - bones_volume
            print(f"✅ pelvic_bones_3mm created: {bones_3mm_volume:.2f} ml (expansion: +{expansion_volume:.2f} ml)")
            
            # Add to RT struct with yellow color
            rt_ext.add_roi_from_mask(pelvic_bones_3mm_mask, "pelvic_bones_3mm", color=[255, 255, 0])
            print("   Added pelvic_bones_3mm to RT struct")
            
            print(f"\n📊 PELVIC BONES SUMMARY:")
            print(f"   pelvic_bones: {bones_volume:.2f} ml")
            print(f"   pelvic_bones_3mm: {bones_3mm_volume:.2f} ml")
            print(f"   3mm expansion volume: {expansion_volume:.2f} ml")
            
        else:
            print("❌ Failed to create pelvic_bones_3mm expansion")
            
    else:
        print("❌ Failed to create pelvic_bones union")
        
else:
    print("❌ No pelvic bone ROIs available")

# Save the enhanced RT struct with all new structures
print("💾 Saving enhanced RT struct...")

# Define the output path
output_path = "/projekty/Odbytnice/Data_Organized/172543/TotalSegmentator_ext.dcm"

try:
    # Save the RT struct with all modifications
    rt_ext.rtstruct.save(output_path)
    
    print(f"✅ Successfully saved enhanced RT struct to:")
    print(f"   {output_path}")
    
    # Provide summary of what was saved
    total_rois = len(rt_ext.rtstruct.get_roi_names())
    fixed_rois = len(rt_ext.get_fixed_rois())
    
    print(f"\n📊 SAVED CONTENT SUMMARY:")
    print(f"   Total ROIs: {total_rois}")
    print(f"   Fixed ROIs in memory: {fixed_rois}")
    
    # List the new structures we created
    new_structures = ["iliac_vess", "iliac_area", "pelvic_bones", "pelvic_bones_3mm"]
    created_structures = []
    
    for struct in new_structures:
        try:
            volume = rt_ext.calculate_volume_ml(struct)
            created_structures.append(f"{struct} ({volume:.1f} ml)")
        except:
            pass
    
    if created_structures:
        print(f"   New structures: {len(created_structures)}")
        for struct in created_structures:
            print(f"     • {struct}")

    # Optional: Verify file size
    import os
    if os.path.exists(output_path):
        file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
        print(f"   File size: {file_size:.1f} MB")
    
except Exception as e:
    print(f"❌ Error saving RT struct: {e}")

In [39]:
import os
import glob
import numpy as np
from pathlib import Path
import logging
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'rt_processing_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def process_patient_rt_structures(patient_dir, base_path="/projekty/Odbytnice/Data_Organized"):
    """
    Process RT structures for a single patient.
    
    Parameters:
    -----------
    patient_dir : str
        Patient directory name (e.g., "172543")
    base_path : str
        Base path containing patient directories
        
    Returns:
    --------
    dict : Processing results and statistics
    """
    patient_id = os.path.basename(patient_dir)
    logger.info(f"🏥 Processing patient: {patient_id}")
    
    # Define paths for this patient
    patient_path = os.path.join(base_path, patient_id)
    ct_dicom_path = os.path.join(patient_path, "CT_DICOM")
    rt_struct_path = os.path.join(patient_path, "TotalSegmentator.dcm")
    output_path = os.path.join(patient_path, "TotalSegmentator_ext.dcm")
    
    # Check if required files/directories exist
    if not os.path.exists(ct_dicom_path):
        logger.error(f"❌ CT_DICOM directory not found: {ct_dicom_path}")
        return {"status": "error", "message": "CT_DICOM directory missing"}
        
    if not os.path.exists(rt_struct_path):
        logger.error(f"❌ TotalSegmentator.dcm not found: {rt_struct_path}")
        return {"status": "error", "message": "TotalSegmentator.dcm missing"}
    
    try:
        # Create RTStructExtended object
        rt_ext = RTStructExtended.create_from(
            dicom_series_path=ct_dicom_path,
            rt_struct_path=rt_struct_path
        )
        rt_ext.fix_failed_rois()
        
        results = {
            "status": "success",
            "patient_id": patient_id,
            "structures_created": [],
            "volumes": {},
            "errors": []
        }
        
        # === ILIAC VESSEL STRUCTURES ===
        logger.info("🔧 Creating iliac vessel structures...")
        
        iliac_vessel_rois = ['iliac_artery_left', 'iliac_artery_right', 'iliac_vena_left', 'iliac_vena_right']
        
        logger.info(f"1️⃣ Creating iliac_vess from: {iliac_vessel_rois}")
        iliac_vess_mask = rt_ext.multi_union(iliac_vessel_rois, auto_fix=True)
        
        if iliac_vess_mask is not None:
            # Calculate volume
            vess_volume = np.sum(iliac_vess_mask) * rt_ext.get_voxel_volume() / 1000.0
            logger.info(f"✅ iliac_vess created: {vess_volume:.2f} ml")
            
            # Add to RT struct with blue color
            rt_ext.add_roi_from_mask(iliac_vess_mask, "iliac_vess", color=[0, 0, 255])
            results["structures_created"].append("iliac_vess")
            results["volumes"]["iliac_vess"] = vess_volume
            
            # Create iliac_area: 7mm expansion from iliac_vess
            logger.info("2️⃣ Creating iliac_area: 7mm expansion from iliac_vess")
            iliac_area_mask = rt_ext.expand_isotropic("iliac_vess", 7.0)
            
            if iliac_area_mask is not None:
                # Calculate volume
                area_volume = np.sum(iliac_area_mask) * rt_ext.get_voxel_volume() / 1000.0
                expansion_volume = area_volume - vess_volume
                logger.info(f"✅ iliac_area created: {area_volume:.2f} ml (expansion: +{expansion_volume:.2f} ml)")
                
                # Add to RT struct with cyan color
                rt_ext.add_roi_from_mask(iliac_area_mask, "iliac_area", color=[0, 255, 255])
                results["structures_created"].append("iliac_area")
                results["volumes"]["iliac_area"] = area_volume
                results["volumes"]["iliac_expansion"] = expansion_volume
                
            else:
                logger.warning("❌ Failed to create iliac_area expansion")
                results["errors"].append("Failed to create iliac_area expansion")
                
        else:
            logger.warning("❌ Failed to create iliac_vess union")
            results["errors"].append("Failed to create iliac_vess union")
            # Log which ROIs are missing
            for roi in iliac_vessel_rois:
                status = "Available" if rt_ext.is_roi_valid(roi) or rt_ext.is_roi_fixed(roi) else "Missing"
                logger.info(f"     {roi}: {status}")
        
        # === PELVIC BONES STRUCTURES ===
        logger.info("🔧 Creating pelvic bones structures...")
        
        pelvic_bone_rois = [
            'sacrum',           # Sacrum
            'hip_left',         # Left hip bone
            'hip_right',        # Right hip bone
            'vertebrae_S1'      # S1 vertebra
        ]
        
        logger.info(f"3️⃣ Creating pelvic_bones from: {pelvic_bone_rois}")
        
        # Check availability of pelvic bone ROIs
        available_bones = []
        for roi in pelvic_bone_rois:
            if rt_ext.is_roi_valid(roi) or rt_ext.is_roi_fixed(roi):
                available_bones.append(roi)
                logger.info(f"     {roi}: Available")
            else:
                logger.info(f"     {roi}: Missing")
        
        if available_bones:
            logger.info(f"   Using {len(available_bones)}/{len(pelvic_bone_rois)} available bone ROIs")
            
            # Create union of all available pelvic bones
            pelvic_bones_mask = rt_ext.multi_union(available_bones, auto_fix=True)
            
            if pelvic_bones_mask is not None:
                # Calculate volume
                bones_volume = np.sum(pelvic_bones_mask) * rt_ext.get_voxel_volume() / 1000.0
                logger.info(f"✅ pelvic_bones created: {bones_volume:.2f} ml")
                
                # Add to RT struct with orange color
                rt_ext.add_roi_from_mask(pelvic_bones_mask, "pelvic_bones", color=[255, 165, 0])
                results["structures_created"].append("pelvic_bones")
                results["volumes"]["pelvic_bones"] = bones_volume
                
                # Create pelvic_bones_3mm: 3mm expansion from pelvic_bones
                logger.info("4️⃣ Creating pelvic_bones_3mm: 3mm expansion from pelvic_bones")
                pelvic_bones_3mm_mask = rt_ext.expand_isotropic("pelvic_bones", 3.0)
                
                if pelvic_bones_3mm_mask is not None:
                    # Calculate volume
                    bones_3mm_volume = np.sum(pelvic_bones_3mm_mask) * rt_ext.get_voxel_volume() / 1000.0
                    expansion_volume = bones_3mm_volume - bones_volume
                    logger.info(f"✅ pelvic_bones_3mm created: {bones_3mm_volume:.2f} ml (expansion: +{expansion_volume:.2f} ml)")
                    
                    # Add to RT struct with yellow color
                    rt_ext.add_roi_from_mask(pelvic_bones_3mm_mask, "pelvic_bones_3mm", color=[255, 255, 0])
                    results["structures_created"].append("pelvic_bones_3mm")
                    results["volumes"]["pelvic_bones_3mm"] = bones_3mm_volume
                    results["volumes"]["pelvic_expansion"] = expansion_volume
                    
                else:
                    logger.warning("❌ Failed to create pelvic_bones_3mm expansion")
                    results["errors"].append("Failed to create pelvic_bones_3mm expansion")
                    
            else:
                logger.warning("❌ Failed to create pelvic_bones union")
                results["errors"].append("Failed to create pelvic_bones union")
                
        else:
            logger.warning("❌ No pelvic bone ROIs available")
            results["errors"].append("No pelvic bone ROIs available")
        
        # === SAVE ENHANCED RT STRUCT ===
        logger.info("💾 Saving enhanced RT struct...")
        
        try:
            rt_ext.rtstruct.save(output_path)
            
            logger.info(f"✅ Successfully saved enhanced RT struct to: {output_path}")
            
            # Summary statistics
            total_rois = len(rt_ext.rtstruct.get_roi_names())
            fixed_rois = len(rt_ext.get_fixed_rois())
            
            results["total_rois"] = total_rois
            results["fixed_rois"] = fixed_rois
            results["output_path"] = output_path
            
            # File size info
            if os.path.exists(output_path):
                file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
                results["file_size_mb"] = file_size
                logger.info(f"   File size: {file_size:.1f} MB")
            
            logger.info(f"📊 PATIENT {patient_id} SUMMARY:")
            logger.info(f"   Total ROIs: {total_rois}")
            logger.info(f"   Fixed ROIs: {fixed_rois}")
            logger.info(f"   New structures: {len(results['structures_created'])}")
            for struct in results["structures_created"]:
                if struct in results["volumes"]:
                    logger.info(f"     • {struct}: {results['volumes'][struct]:.1f} ml")
            
        except Exception as e:
            logger.error(f"❌ Error saving RT struct: {e}")
            results["status"] = "partial_success"
            results["errors"].append(f"Save error: {str(e)}")
        
        return results
        
    except Exception as e:
        logger.error(f"❌ Error processing patient {patient_id}: {e}")
        return {
            "status": "error", 
            "patient_id": patient_id,
            "message": f"Processing error: {str(e)}"
        }

def process_all_patients(base_path="/projekty/Odbytnice/Data_Organized"):
    """
    Process RT structures for all patients in the base directory.
    
    Parameters:
    -----------
    base_path : str
        Base path containing patient directories
        
    Returns:
    --------
    dict : Overall processing summary and individual results
    """
    logger.info("🚀 Starting batch processing of RT structures...")
    logger.info(f"📁 Base directory: {base_path}")
    
    # Find all patient directories
    if not os.path.exists(base_path):
        logger.error(f"❌ Base directory does not exist: {base_path}")
        return {"status": "error", "message": "Base directory not found"}
    
    # Get all subdirectories (patient IDs)
    patient_dirs = [d for d in os.listdir(base_path) 
                   if os.path.isdir(os.path.join(base_path, d))]
    
    if not patient_dirs:
        logger.warning("❌ No patient directories found")
        return {"status": "error", "message": "No patient directories found"}
    
    logger.info(f"📊 Found {len(patient_dirs)} patient directories")
    
    # Initialize summary statistics
    summary = {
        "total_patients": len(patient_dirs),
        "successful": 0,
        "partial_success": 0,
        "failed": 0,
        "results": [],
        "errors": []
    }
    
    # Process each patient
    for i, patient_dir in enumerate(sorted(patient_dirs), 1):
        logger.info(f"\n{'='*60}")
        logger.info(f"Processing patient {i}/{len(patient_dirs)}: {patient_dir}")
        logger.info(f"{'='*60}")
        
        try:
            result = process_patient_rt_structures(patient_dir, base_path)
            summary["results"].append(result)
            
            # Update counters
            if result["status"] == "success":
                summary["successful"] += 1
            elif result["status"] == "partial_success":
                summary["partial_success"] += 1
            else:
                summary["failed"] += 1
                summary["errors"].append(f"Patient {patient_dir}: {result.get('message', 'Unknown error')}")
                
        except Exception as e:
            logger.error(f"❌ Unexpected error processing patient {patient_dir}: {e}")
            summary["failed"] += 1
            summary["errors"].append(f"Patient {patient_dir}: Unexpected error - {str(e)}")
    
    # Final summary
    logger.info(f"\n{'='*60}")
    logger.info("🏁 BATCH PROCESSING COMPLETE")
    logger.info(f"{'='*60}")
    logger.info(f"📊 FINAL SUMMARY:")
    logger.info(f"   Total patients: {summary['total_patients']}")
    logger.info(f"   ✅ Successful: {summary['successful']}")
    logger.info(f"   ⚠️  Partial success: {summary['partial_success']}")
    logger.info(f"   ❌ Failed: {summary['failed']}")
    
    if summary["errors"]:
        logger.info(f"\n🚨 ERRORS ENCOUNTERED:")
        for error in summary["errors"]:
            logger.error(f"   • {error}")
    
    # Calculate overall statistics
    successful_results = [r for r in summary["results"] if r["status"] in ["success", "partial_success"]]
    if successful_results:
        logger.info(f"\n📈 AGGREGATE STATISTICS:")
        
        # Count structure creation success rates
        structure_counts = {}
        for result in successful_results:
            for struct in result.get("structures_created", []):
                structure_counts[struct] = structure_counts.get(struct, 0) + 1
        
        for struct, count in structure_counts.items():
            success_rate = (count / len(successful_results)) * 100
            logger.info(f"   {struct}: {count}/{len(successful_results)} ({success_rate:.1f}%)")
    
    return summary

# Main execution
if __name__ == "__main__":
    # Run the batch processing
    results = process_all_patients()
    
    # Optionally save results to a file for further analysis
    import json
    output_file = f"rt_processing_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    # Make results JSON serializable
    json_results = {
        "summary": {k: v for k, v in results.items() if k != "results"},
        "patient_count": len(results.get("results", [])),
        "processing_date": datetime.now().isoformat()
    }
    
    try:
        with open(output_file, 'w') as f:
            json.dump(json_results, f, indent=2)
        logger.info(f"📁 Results summary saved to: {output_file}")
    except Exception as e:
        logger.warning(f"⚠️ Could not save results to JSON: {e}")
    
    print(f"\n🎯 Processing complete. Check the log file for detailed information.")

2025-07-06 21:46:25,981 - INFO - 🚀 Starting batch processing of RT structures...
2025-07-06 21:46:25,981 - INFO - 📁 Base directory: /projekty/Odbytnice/Data_Organized
2025-07-06 21:46:26,018 - INFO - 📊 Found 206 patient directories
2025-07-06 21:46:26,019 - INFO - 
2025-07-06 21:46:26,021 - INFO - Processing patient 1/206: 172543
2025-07-06 21:46:26,022 - INFO - ============================================================
2025-07-06 21:46:26,023 - INFO - 🏥 Processing patient: 172543


🔍 Auto-detecting failed ROIs...
Error getting mask for ROI 'kidney_left': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'liver': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'lung_lower_lobe_right': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'colon': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for ROI 'sacrum': OpenCV(4.11.0) /io/opencv/modules/imgproc/src/drawing.cpp:2426: error: (-215:Assertion failed) p.checkVector(2, CV_32S) > 0 in function 'fillPoly'

Error getting mask for RO

2025-07-06 21:46:34,532 - INFO - 🔧 Creating iliac vessel structures...
2025-07-06 21:46:34,534 - INFO - 1️⃣ Creating iliac_vess from: ['iliac_artery_left', 'iliac_artery_right', 'iliac_vena_left', 'iliac_vena_right']


✅ Fixed: rib_right_12 → 165 voxels (0.68 ml)
   Stored in memory: rib_right_12
🔧 Fixing ROI: costal_cartilages
✅ Fixed: costal_cartilages → 5,700 voxels (23.48 ml)
   Stored in memory: costal_cartilages

📊 RESULTS:
✅ Successfully recovered: 14/14 ROIs
❌ Still failed: 0 ROIs
🔢 Total recovered voxels: 632,445
💾 Fixed ROIs stored in memory and ready to use

✅ RECOVERED ROIS:
  - kidney_left
  - liver
  - lung_lower_lobe_right
  - colon
  - sacrum
  - vertebrae_L5
  - vertebrae_T12
  - iliac_artery_left
  - femur_left
  - iliopsoas_left
  - skull
  - rib_left_12
  - rib_right_12
  - costal_cartilages


2025-07-06 21:46:34,919 - INFO - ✅ iliac_vess created: 90.30 ml
2025-07-06 21:46:35,083 - INFO - 2️⃣ Creating iliac_area: 7mm expansion from iliac_vess
2025-07-06 21:46:40,950 - INFO - ✅ iliac_area created: 355.00 ml (expansion: +264.69 ml)
2025-07-06 21:46:41,122 - INFO - 🔧 Creating pelvic bones structures...
2025-07-06 21:46:41,123 - INFO - 3️⃣ Creating pelvic_bones from: ['sacrum', 'hip_left', 'hip_right', 'vertebrae_S1']
2025-07-06 21:46:41,123 - INFO -      sacrum: Available
2025-07-06 21:46:41,271 - INFO -      hip_left: Available
2025-07-06 21:46:41,421 - INFO -      hip_right: Available
2025-07-06 21:46:41,482 - INFO -      vertebrae_S1: Available
2025-07-06 21:46:41,483 - INFO -    Using 4/4 available bone ROIs
2025-07-06 21:46:41,876 - INFO - ✅ pelvic_bones created: 1158.43 ml
2025-07-06 21:46:42,118 - INFO - 4️⃣ Creating pelvic_bones_3mm: 3mm expansion from pelvic_bones
2025-07-06 21:46:43,080 - INFO - ✅ pelvic_bones_3mm created: 1710.28 ml (expansion: +551.85 ml)
2025-07-06

...

✅ Fixed: costal_cartilages → 5,505 voxels (15.75 ml)
   Stored in memory: costal_cartilages

📊 RESULTS:
✅ Successfully recovered: 4/4 ROIs
❌ Still failed: 0 ROIs
🔢 Total recovered voxels: 205,828
💾 Fixed ROIs stored in memory and ready to use

✅ RECOVERED ROIS:
  - small_bowel
  - vertebrae_S1
  - vertebrae_L2
  - costal_cartilages


2025-07-06 22:58:54,950 - INFO - ✅ iliac_vess created: 93.58 ml
2025-07-06 22:58:55,095 - INFO - 2️⃣ Creating iliac_area: 7mm expansion from iliac_vess
2025-07-06 22:59:00,944 - INFO - ✅ iliac_area created: 324.35 ml (expansion: +230.77 ml)
2025-07-06 22:59:01,068 - INFO - 🔧 Creating pelvic bones structures...
2025-07-06 22:59:01,069 - INFO - 3️⃣ Creating pelvic_bones from: ['sacrum', 'hip_left', 'hip_right', 'vertebrae_S1']
2025-07-06 22:59:01,181 - INFO -      sacrum: Available
2025-07-06 22:59:01,300 - INFO -      hip_left: Available
2025-07-06 22:59:01,423 - INFO -      hip_right: Available
2025-07-06 22:59:01,423 - INFO -      vertebrae_S1: Available
2025-07-06 22:59:01,424 - INFO -    Using 4/4 available bone ROIs
2025-07-06 22:59:01,839 - INFO - ✅ pelvic_bones created: 843.96 ml
2025-07-06 22:59:02,407 - INFO - 4️⃣ Creating pelvic_bones_3mm: 3mm expansion from pelvic_bones
2025-07-06 22:59:03,228 - INFO - ✅ pelvic_bones_3mm created: 1251.89 ml (expansion: +407.92 ml)
2025-07-06 

Writing file to /projekty/Odbytnice/Data_Organized/480011/TotalSegmentator_ext.dcm


2025-07-06 22:59:03,819 - INFO - ✅ Successfully saved enhanced RT struct to: /projekty/Odbytnice/Data_Organized/480011/TotalSegmentator_ext.dcm
2025-07-06 22:59:03,820 - INFO -    File size: 7.1 MB
2025-07-06 22:59:03,820 - INFO - 📊 PATIENT 480011 SUMMARY:
2025-07-06 22:59:03,820 - INFO -    Total ROIs: 55
2025-07-06 22:59:03,821 - INFO -    Fixed ROIs: 4
2025-07-06 22:59:03,821 - INFO -    New structures: 4
2025-07-06 22:59:03,821 - INFO -      • iliac_vess: 93.6 ml
2025-07-06 22:59:03,821 - INFO -      • iliac_area: 324.4 ml
2025-07-06 22:59:03,821 - INFO -      • pelvic_bones: 844.0 ml
2025-07-06 22:59:03,821 - INFO -      • pelvic_bones_3mm: 1251.9 ml
2025-07-06 22:59:03,844 - INFO - 
2025-07-06 22:59:03,845 - INFO - Processing patient 206/206: root
2025-07-06 22:59:03,845 - INFO - ============================================================
2025-07-06 22:59:03,845 - INFO - 🏥 Processing patient: root
2025-07-06 22:59:03,846 - ERROR - ❌ CT_DICOM directory not found: /projekty/Odbytn


🎯 Processing complete. Check the log file for detailed information.
